MLflow setup:

Tracking server: yes, remote server (EC2).
Backend store: postgresql database.
Artifacts store: s3 bucket.
The experiments can be explored by accessing the remote server.

The exampe uses AWS to host a remote server. In order to run the example you'll need an AWS account. Follow the steps described in the file mlflow_on_aws.md to create a new AWS account and launch the tracking server.

In [5]:
import os
import boto3

# Use the environment variables
session = boto3.Session(
    aws_access_key_id=os.environ.get('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.environ.get('AWS_SECRET_ACCESS_KEY'),
    aws_session_token=os.environ.get('AWS_SESSION_TOKEN'),  # Optional
    region_name=os.environ.get('AWS_DEFAULT_REGION')
)

# Example: Create an S3 client using the session
s3 = session.client('s3')

# List all buckets in S3
response = s3.list_buckets()
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  mlflow-s3-artifact-remote


In [6]:
import mlflow
import os

#os.environ["AWS_PROFILE"] = "" # fill in with your AWS profile. More info: https://docs.aws.amazon.com/sdk-for-java/latest/developer-guide/setup.html#setup-credentials

# # Set the environment variables for AWS credentials and region
# os.environ['AWS_ACCESS_KEY_ID'] = 'your_access_key_id'
# os.environ['AWS_SECRET_ACCESS_KEY'] = 'your_secret_access_key'
# os.environ['AWS_SESSION_TOKEN'] = 'your_session_token'  # Optional, only needed for temporary credentials
# os.environ['AWS_DEFAULT_REGION'] = 'your_region'

TRACKING_SERVER_HOST = "ec2-18-153-105-179.eu-central-1.compute.amazonaws.com" # fill in with the public DNS of the EC2 instance
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [7]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://ec2-18-153-105-179.eu-central-1.compute.amazonaws.com:5000'


In [8]:
mlflow.search_experiments()

[<Experiment: artifact_location='s3://mlflow-s3-artifact-remote/0', creation_time=1716721354183, experiment_id='0', last_update_time=1716721354183, lifecycle_stage='active', name='Default', tags={}>]

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2024/05/26 13:07:31 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.


default artifacts URI: 's3://mlflow-s3-artifact-remote/1/885738a8f09e44d1b65ade3f55b9a35b/artifacts'


In [11]:
mlflow.search_experiments()

[<Experiment: artifact_location='/Users/rajitsanghvi/Library/CloudStorage/OneDrive-OPEDGMBH/General/01_Github/MLOps/02-experiment-tracking/running_mlflow-example/artifacts_local/1', creation_time=1716560457982, experiment_id='1', last_update_time=1716560457982, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1716559949916, experiment_id='0', last_update_time=1716559949916, lifecycle_stage='active', name='Default', tags={}>]

In [10]:
from mlflow.tracking import MlflowClient

client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [11]:
client.search_registered_models()

[]

In [12]:
client.search_experiments()

[<Experiment: artifact_location='s3://mlflow-s3-artifact-remote/1', creation_time=1716721652053, experiment_id='1', last_update_time=1716721652053, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='s3://mlflow-s3-artifact-remote/0', creation_time=1716721354183, experiment_id='0', last_update_time=1716721354183, lifecycle_stage='active', name='Default', tags={}>]

In [37]:
experiment = client.get_experiment_by_name("my-experiment-1")
runs = client.search_runs(
        experiment_ids=experiment.experiment_id)
runs[0]

<Run: data=<RunData: metrics={'accuracy': 0.96}, params={'C': '0.1', 'random_state': '42'}, tags={'mlflow.log-model.history': '[{"run_id": "11d9c9f41cc2454a9077070d564d214a", '
                             '"artifact_path": "models", "utc_time_created": '
                             '"2024-05-24 14:20:58.518788", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"predict_fn": "predict", "loader_module": '
                             '"mlflow.sklearn", "python_version": "3.9.19", '
                             '"env": {"conda": "conda.yaml", "virtualenv": '
                             '"python_env.yaml"}}, "sklearn": '
                             '{"pickled_model": "model.pkl", '
                             '"sklearn_version": "1.4.2", '
                             '"serialization_format": "cloudpickle", "code": '
                             'null}}, "model_uuid": '
                             '"d62e8ec1176

In [14]:
run_id = "885738a8f09e44d1b65ade3f55b9a35b"
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Registered model 'iris-classifier' already exists. Creating a new version of this model...
2024/05/26 13:12:36 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1716721956903, current_stage='None', description='', last_updated_timestamp=1716721956903, name='iris-classifier', run_id='885738a8f09e44d1b65ade3f55b9a35b', run_link='', source='s3://mlflow-s3-artifact-remote/1/885738a8f09e44d1b65ade3f55b9a35b/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>

In [15]:
model_name = "iris-classifier"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: None


/var/folders/b4/lg6cghr91lg7q_c13xh443mw0000gn/T/ipykernel_69826/2355489280.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [17]:
model_version = 1
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/var/folders/b4/lg6cghr91lg7q_c13xh443mw0000gn/T/ipykernel_69826/1600074043.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1716721956903, current_stage='Staging', description='', last_updated_timestamp=1716722077318, name='iris-classifier', run_id='885738a8f09e44d1b65ade3f55b9a35b', run_link='', source='s3://mlflow-s3-artifact-remote/1/885738a8f09e44d1b65ade3f55b9a35b/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>